# A meta-analysis of the mechanical properties of ice-templated ceramics and metals

Welcome to the notebook accompanying this paper: "A meta-analysis of the mechanical properties of ice-templated ceramics and metals", published in *Science and Technology of Advanced Materials* (link coming as soon as the paper is available online).

Follow through the notebook to generate the plots published in the paper. You need to have *Matplotlib*, *Numpy*, and *Pandas* installed. If you are not familiar with the IPython notebook format, start here: <http://ipython.org/index.html>

Your files and directories should be organized as follow, in your working directory:

* *code*: contains this IPython notebook
* *figures*: where all figures will be saved
* *data*: where the CSV files should be. There are two sets of data: 
    * The data compiled from the literature review on ice templating (*icetemplating.csv*). 
    * The data corresponding to the properties of cellular ceramics (*studart.csv*), and compiled in this paper: Studart, A. R., Gonzenbach, U. T., Tervoort, E., & Gauckler, L. J. (2006). Processing Routes to Macroporous Ceramics: A Review. Journal of the American Ceramic Society, 89(6), 1771–1789. doi:10.1111/j.1551-2916.2006.01044.x The paper can be found here <http://www.blackwell-synergy.com/doi/abs/10.1111/j.1551-2916.2006.01044.x> (paywall)

    You can download these sets of data from Figshare here: <http://dx.doi.org/10.6084/m9.figshare.1412626>

-May, 2015. Sylvain Deville, Sylvain Meille, Jordi Seuba

Email to <sylvaindeville@gmail.com> or tweet to <http://twitter.com/DevilleSy> if you have any questions or comments, or just want to say hi!

In [3]:
import pandas as pd

In [4]:
import matplotlib.pyplot as plt

In [5]:
import io

In [6]:
from numpy import linspace
from matplotlib import rc #for latex use on the plots
from matplotlib.ticker import MultipleLocator, FormatStrFormatter # for labels formattings
%matplotlib inline

In [7]:
perso_linewidth = 0.3
# This will change your default rcParams
def init_plotting():
    plt.rcParams['figure.figsize'] = (3,3)
    plt.rcParams['font.size'] = 8
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['axes.titlesize'] = 1.5*plt.rcParams['font.size']
    plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']
    plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']
#     plt.rcParams['savefig.dpi'] = int(2)*plt.rcParams['savefig.dpi']
    plt.rcParams['axes.linewidth'] = perso_linewidth
    plt.rcParams['savefig.dpi'] = '300'
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['axes.edgecolor'] = '0'
    plt.rcParams['axes.grid'] = False
    plt.rcParams['grid.color']='white'
    plt.rcParams['grid.linestyle'] = '-'
    plt.rcParams['grid.linewidth'] = '0.1'
    plt.rcParams['axes.axisbelow'] = True
    plt.rcParams['lines.markersize']= 2.3 
    plt.rcParams['lines.markeredgewidth']= '0.1'
    plt.rcParams['lines.color']= 'r' 
    plt.rcParams['lines.marker']= 'o' 
    plt.rcParams['lines.linestyle']= '' 
    plt.rcParams['xtick.color']= '0'
    plt.rcParams['ytick.color']= '0'
#     plt.rcParams['axes.color_cycle']= ['#3778bf', '#feb308', '#a8a495', '#7bb274', '#825f87']
    plt.gca().spines['right'].set_color('none')
    plt.gca().spines['right'].set_visible('False')
    plt.gca().spines['top'].set_visible('False')
    plt.gca().spines['top'].set_color('none')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.gca().yaxis.set_ticks_position('left')
    plt.rcParams['ytick.minor.size']= 1.5
    plt.rcParams['ytick.major.width']= perso_linewidth
    plt.rcParams['ytick.minor.width']= perso_linewidth
    plt.rcParams['xtick.major.width']= perso_linewidth
    plt.rcParams['xtick.minor.width']= perso_linewidth

init_plotting()

plt.close() # this line to avoid an empty plot showing up

In [8]:
# function to list the bib key of papers used in the plots
# The output is formatted to be used directly in Markdown. Pandoc can take care of the reference list from here.
def list_authors(data):
    authors = data.dropna()
    authors = authors.drop_duplicates()
    authors.tolist()
    print (str(len(authors)) + " papers")
    print ('@'+'; @'.join(authors))

In [9]:
# common formatting for the plots, not passed as rcParams
figuresize = 3
titlefontsize = 8
titleweight = 'bold'

rc('font', **{'family':'serif', 'serif':['Palatino']})
rc('text', usetex=True)

# Mechanical models used to predict the compressive strength

## Closed cells
$$\sigma=\sigma_{p}\left(C_6\left(\phi\frac{\rho^*}{\rho_s}\right)+C_{6}^{''}\left(1-\phi\right)\frac{\rho^*}{\rho_s}\right) \text{  (eq. 1)}$$ 
with $C_6=0.65$ and $C_6^"=1$, and where $\phi$ is the solid fraction in the edges, $\rho_s$ and $\rho^*$ the apparent density of the dense and cellular material respectively, and $\sigma_p$ the modulus of rupture of the solid cell wall material.

## Open cell, brittle crushing model
$$\sigma=\sigma_{p}C_4\left(\frac{\rho^*}{\rho_s}\right)^{3/2}\text{  (eq. 2)}$$ 
with $C_4=0.2$, and where $\sigma_{p}$ is the modulus of rupture of the solid cell wall material, and $\rho_s$ and $\rho^*$ the apparent density of the dense and cellular material respectively.

## Honeycomb, out of plane
$$\sigma=6 E_s \left(\frac{\rho^*}{\rho_s}\right)^3 \text{  (eq. 3)}$$
where $E_s$ the Young's modulus of the corresponding dense material, and $\rho_s$ and $\rho^*$ the apparent density of the dense and cellular material respectively.

In [10]:
# Parameters of the Ashby model
phi_edges = 0
C2 = 0.2
C6 = 0.65
C6s = 1

# data for the upper bound value of compressive strength vs porosity
relative_density_table = linspace(0.0, 1.0, num=1000)
relative_porosity = [(1-i)*100 for i in relative_density_table]

# open cells model:
brit_crush_str = [C2*i**(1.5) for i in relative_density_table]

# closed cells models:
closedcell_str = [C6*phi_edges*i +C6s*(1-phi_edges)*i for i in relative_density_table]

# functions used to plot the prediction of the models on the plots
def plot_upper_limit():
        plt.plot(relative_porosity, brit_crush_str, linestyle=':', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round')
        plt.plot(relative_porosity, closedcell_str, linestyle=':', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round')
        
def plot_upper_limit_log():
        plt.plot(relative_density_table, brit_crush_str, linestyle=':', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round')
        plt.plot(relative_density_table, closedcell_str, linestyle=':', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round')        

strength_table = linspace(0.1, 1000.0, num=10000)
        
def plot_upper_limit_honeycomb():
        plt.plot(strength_table, strength_table, linestyle='--', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round', markevery='2')    


# Load the data

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [12]:
df = pd.read_csv("CSs_dados_tratados_002.csv", )
df

,Device,Size [cm2],Temperature [C],Irradiance [W/m2],Jsc [mA/cm2],Isc [mA],Voc [mV],Impp [mA],Vmpp [mV],Pmpp [mW],...,f2,f5,f8,f10,Ref GaAs,QDSKmeio,QDSKfundo,QDSK,I+,Si
0,3906,0.035387,25.360,995.050732,16.742954,0.592484,882.761335,0.520127,714.941595,0.371861,...,0,0,0,0,0,0,0,0,0,0
1,3906,0.037082,25.465,995.050732,16.809543,0.623330,903.100750,0.569063,738.691294,0.420362,...,0,0,0,0,0,0,0,0,0,0
2,3906,0.036205,25.405,995.050732,16.808316,0.608537,905.901882,0.567865,748.108530,0.424825,...,0,0,0,0,0,0,0,0,0,0
3,3906,0.102450,25.000,990.831707,12.553037,1.286059,882.503050,1.179564,732.329341,0.863829,...,0,0,0,0,0,0,0,0,0,0
4,3906,0.102450,25.000,990.831707,12.436932,1.274164,883.127060,1.168964,734.897123,0.859068,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,3975,0.048300,25.000,998.034634,16.977690,0.820022,945.689372,0.772679,810.879477,0.626549,...,0,0,1,0,0,0,0,0,0,0
2766,3975,0.048300,25.000,998.034634,17.139725,0.827849,946.851068,0.778047,808.880230,0.629347,...,0,0,1,0,0,0,0,0,0,0
2767,3975,0.048300,25.000,998.034634,17.103985,0.826122,933.315074,0.774141,785.604613,0.608169,...,0,0,1,0,0,0,0,0,0,0
2768,3975,0.048300,25.000,998.034634,17.129158,0.827338,936.091055,0.775803,786.010630,0.609789,...,0,0,1,0,0,0,0,0,0,0


In [15]:
# data = pd.io.parsers.read_csv('icetemplating.csv', sep=';', decimal=',')
data = pd.io.parsers.read_csv('icetemplating.csv', sep=';', decimal=',')
data

,Mendeley key,Citation,Solvant,Material,Material class,Ceramic class,Constant temperature,Cooling rate,Interface velocity,Wall thickness,...,Youngs Modulus (MPa),Normalized Young Modulus,green body ?,Compressive strength dense material,Flexural strength dense material,Normalized strength (compression),Normalized strength (flexion),E_dense,Honeycomb_Ashby,Normalized_strength_Elastic_modulus
0,Shen2014a,"Shen, P., Xi, J., Fu, Y., Shaga, A., Sun, C., ...",water,alumina,cera,oxide,NaN,NaN,NaN,10-71,...,NaN,NaN,0.0,2600.0,400.0,0.017500,NaN,390.0,69.71094,0.017949
1,Shen2014a,"Shen, P., Xi, J., Fu, Y., Shaga, A., Sun, C., ...",water,alumina,cera,oxide,NaN,NaN,NaN,8-58,...,NaN,NaN,0.0,2600.0,400.0,0.055000,NaN,390.0,91.97136,0.056410
2,Shen2014a,"Shen, P., Xi, J., Fu, Y., Shaga, A., Sun, C., ...",water,alumina,cera,oxide,NaN,NaN,NaN,10-72,...,NaN,NaN,0.0,2600.0,400.0,0.160000,0.00500,390.0,161.27514,0.164103
3,Yang2014,"Yang, H., Ye, F., Liu, Q., Liu, S., Gao, Y., &...",water,silicon nitride+alumina+yttria,cera,covalent,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,3200.0,1000.0,NaN,0.00087,NaN,NaN,NaN
4,Yang2014,"Yang, H., Ye, F., Liu, Q., Liu, S., Gao, Y., &...",water,silicon nitride+alumina+yttria,cera,covalent,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,3200.0,1000.0,NaN,0.00112,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,Li2015b,"Li, W., Porter, M. M., Olevsky, E. A., German,...",NaN,titania,cera,oxide,NaN,10.0,NaN,NaN,...,NaN,NaN,0.0,700.0,400.0,0.004700,NaN,220.0,2.90004,0.008545
662,Li2015b,"Li, W., Porter, M. M., Olevsky, E. A., German,...",NaN,titania,cera,oxide,NaN,10.0,NaN,NaN,...,NaN,NaN,0.0,700.0,400.0,0.002825,NaN,220.0,4.45500,0.005136
663,Li2015b,"Li, W., Porter, M. M., Olevsky, E. A., German,...",NaN,titania,cera,oxide,NaN,10.0,NaN,NaN,...,NaN,NaN,0.0,700.0,400.0,0.009700,NaN,220.0,20.62500,0.017636
664,Li2015b,"Li, W., Porter, M. M., Olevsky, E. A., German,...",NaN,titania,cera,oxide,NaN,10.0,NaN,NaN,...,NaN,NaN,0.0,700.0,400.0,1.285000,NaN,220.0,492.68736,2.336364


In [16]:
foams = pd.io.parsers.read_csv('studart.csv', sep=';', decimal=',')

FileNotFoundError: [Errno 2] No such file or directory: 'studart.csv'

# Create a few subsets of data

In [ ]:
water = data[(data['Solvant'] == 'water') & (data['green body ?'] < 1)]
camphene = data[(data['Solvant'] == 'camphene')]
tba = data[(data['Solvant'] == 'TBA')]

cellular = data[(data['failure mode'] == 'cellular')]
brittle = data[(data['failure mode'] == 'brittle')]
failureunknown = data[(data['failure mode'] == '?')]

defects = data[(data['Defects'] == 'yes')]
nodefects = data[(data['Defects'] == 'no')]
notsureifdefects = data[(data['Defects'] == '?')]

withparticlesize = data[(data['Granulo poudre'] > 0)]

# Porosity vs. solid loading, for the three most commonly used solvents
Figure 3 of the paper

In [ ]:
from matplotlib.texmanager import TexManager

In [ ]:
import sys, os, math, tempfile, atexit, shutil
from joblib import Parallel, delayed
import matplotlib.texmanager

In [ ]:
TexManager()

In [ ]:
# Subprocess function
def subprocess_task():
    mpldir = tempfile.mkdtemp()
    atexit.register(shutil.rmtree, mpldir)
    umask = os.umask(0)
    os.umask(umask)
    os.chmod(mpldir, 0o777 & ~umask)
    os.environ['HOME'] = mpldir
    os.environ['MPLCONFIGDIR'] = mpldir
    import matplotlib
    class TexManager(matplotlib.texmanager.TexManager):
        texcache = os.path.join(mpldir, 'tex.cache')
    matplotlib.texmanager.TexManager = TexManager
    matplotlib.rcParams['ps.useafm'] = True
    matplotlib.rcParams['pdf.use14corefonts'] = True
    matplotlib.rcParams['text.usetex'] = True

    # From here on, safe to use matplotlib in parallel


# Main process function
def mainprocess_task(n_threads=32):
    import numpy as np
    import matplotlib.pyplot as plt
    with Parallel(n_jobs=n_threads) as parallel:
        parallel(delayed(subprocess_task)() for i in range(0,256))

In [ ]:
subprocess_task()

In [ ]:
mainprocess_task(n_threads=32)

In [ ]:
fig = plt.figure(figsize = (3*figuresize, figuresize))

ax = fig.add_subplot(1,3,1)
plt.plot(water['Solid Loading'], water['Porosity'])
ax.set_xlabel('Solid Loading (vol.\%)')
ax.set_ylabel('Porosity (\%)')
plt.title('Water', fontsize=titlefontsize, weight=titleweight)
plt.xlim(0,65)
plt.ylim(0,100)

ax = fig.add_subplot(1,3,2)
plt.plot(camphene['Solid Loading'], camphene['Porosity'])
ax.set_xlabel('Solid Loading (vol.\%)')
plt.xlim(0,65)
plt.ylim(0,100)
ax.set_yticklabels([])
plt.title('Camphene', fontsize=titlefontsize, weight=titleweight)

ax = fig.add_subplot(1,3,3)
plt.plot(tba['Solid Loading'], tba['Porosity'])
ax.set_xlabel('Solid Loading (vol.\%)')
plt.xlim(0,65)
plt.ylim(0,100)
ax.set_yticklabels([])
plt.title('TBA', fontsize=titlefontsize, weight=titleweight)

plt.tight_layout()
fig.savefig('../figures/figure3.png')

# Range of pore size achieved for the three most commonly used solvents
Figure 4 of the paper

In [ ]:
#extract the data with a pore value
water_porewidth = water[(water['Pore width'] > 0)]
camphene_porewidth = camphene[(camphene['Pore width'] > 0)]
tba_porewidth = tba[(tba['Pore width'] > 0)]

water_porewidth = water['Pore width'].tolist()
camphene_porewidth = camphene['Pore width'].tolist()
tba_porewidth = tba['Pore width'].tolist()

# remove duplicate values
water_porewidth = list(set(water_porewidth))
camphene_porewidth = list(set(camphene_porewidth))
tba_porewidth = list(set(tba_porewidth))

# remove NaN and sort list
water_porewidth = [x for x in water_porewidth if x>0]
water_porewidth.sort()

camphene_porewidth = [x for x in camphene_porewidth if x>0]
camphene_porewidth.sort()

tba_porewidth = [x for x in tba_porewidth if x>0]
tba_porewidth.sort()

# Figure 
fig = plt.figure(figsize=(figuresize, 0.75*figuresize))
ax1 = fig.add_subplot(131)
ax1.plot(water_porewidth)
ax1.set_yscale('log')
ax1.set_ylabel('Pore size (\mu m)')
rectangle = plt.Rectangle((-1, min(water_porewidth)), len(water_porewidth)+1, max(water_porewidth)-min(water_porewidth), color='0.9', zorder=0)
plt.gca().add_patch(rectangle)
plt.ylim(0,500)
plt.xlim(-1,len(water_porewidth))
plt.title('Water', fontsize=titlefontsize)
plt.gca().xaxis.set_major_locator(plt.NullLocator())

ax2 = fig.add_subplot(132, sharey=ax1)
ax2.plot(camphene_porewidth)
ax2.ticklabel_format(axis='both', visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
rectangle = plt.Rectangle((-1, min(camphene_porewidth)), len(camphene_porewidth)+1, max(camphene_porewidth)-min(camphene_porewidth), color='0.9', zorder=0)
plt.gca().add_patch(rectangle)
plt.ylim(0,500)
plt.xlim(-1,len(camphene_porewidth))
plt.title('Camphene', fontsize=titlefontsize)
plt.gca().xaxis.set_major_locator(plt.NullLocator())

ax3 = fig.add_subplot(133, sharey=ax1)
ax3.plot(tba_porewidth)
ax3.ticklabel_format(visible=False)
plt.setp(ax3.get_xticklabels(), visible=False)
plt.setp(ax3.get_yticklabels(), visible=False)
rectangle = plt.Rectangle((-1, min(tba_porewidth)), len(tba_porewidth)+1, max(tba_porewidth)-min(tba_porewidth), color='0.9', zorder=0)
plt.gca().add_patch(rectangle)
plt.ylim(1,1000)
plt.xlim(-1,len(tba_porewidth))
plt.title('TBA', fontsize=titlefontsize)
plt.tight_layout()
fig.subplots_adjust(wspace=0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())

fig.savefig('../figures/figure4.png')

# Pore size vs. total porosity content, all solvents
Figure 5 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(data['Porosity'], data['Pore width'])
ax.set_ylabel('Pore size (\mu m)')
ax.set_xlabel('Porosity (\%)')
#ax.set_yscale('log')
plt.ylim(-10, 520)
plt.xlim(15, 100)
plt.tight_layout()
fig.savefig('../figures/figure5.png')

# Overview of the compressive strength achieved
Figure 6 of the paper

In [ ]:
from scipy.optimize import curve_fit

# function used to fit the regression. Strength is assumed to vary with the power law of the relative density
def func (x, a, b, c):
    return a*(1-x/100)**b+c

fig = plt.figure()

# fit the data with a regression analysis
data_bis = data[(data['Porosity'] > 0) & (data['Normalized strength (compression)'] > 0)]
fitParamsIT, fitCovIT = curve_fit(func, data_bis['Porosity'], data_bis['Normalized strength (compression)'])
porosity_table = linspace(0, 100.0, num=10000)

ax = fig.add_subplot(1,1,1)
plt.plot(data['Porosity'], data['Normalized strength (compression)'])
plt.plot(foams['porosity'], foams['norm strength'])
plt.plot(porosity_table, func(porosity_table, fitParamsIT[0], fitParamsIT[1], fitParamsIT[2]), marker='', linestyle=':', color='k', dashes=[0.3, 2], zorder=1, dash_capstyle='round')
plot_upper_limit() # plot the open and close-cell model predictions
ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_ylabel('Compressive Strength (normalized)')
xmin = 15
xmax = 100
ymin = 0.0001
ymax = 2
plt.ylim(ymin, ymax)
plt.xlim(xmin, xmax)

plt.tight_layout()
fig.savefig('../figures/figure6.png')

In [ ]:
# parameters of the fit
print 'Exponent of the power law: '+str(fitParamsIT[1])

In [ ]:
# Extract the bibliographic references used for the plot
authors = data['Mendeley key']
list_authors(authors)

# Overview of the flexural strength achieved
Figure 7 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(data['Porosity'], data['Normalized strength (flexion)'])
ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_ylabel('Flexural Strength (normalized)')
plt.ylim(0.0001, 2)
plt.xlim(20, 90)

plt.tight_layout()
fig.savefig('../figures/figure7.png')

In [ ]:
# Extract the bib ref of the above plot
dataplot = data[(data['Normalized strength (flexion)'] > 0)]
authors = dataplot['Mendeley key']
list_authors(authors)

# Compressive strength vs. pore size
Figure 8 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(data['Pore width'], data['Normalized strength (compression)'])
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Pore size (\mu m)')
ax.set_ylabel('Compressive Strength (normalized)')
plt.ylim(0.0001, 5)
plt.xlim(2, 700)
plt.tight_layout()

# line with -1/2 slope
poresize_table = linspace(60, 200.0, num=10000)
plt.plot(poresize_table, 100*poresize_table**(-1/2), marker='', linestyle='-', color='black', linewidth='0.3')
plt.plot([100, 120], [1, 1], 'k-', lw=0.3)
plt.annotate('-1/2', (110, 1.2), fontsize='5')
            
fig.savefig('../figures/figure8.png')

In [ ]:
# Extract the bib ref of the above plot
dataplot = data[(data['Pore width'] > 0)]
authors = dataplot['Mendeley key']
list_authors(authors)

# Compressive strength and failure modes
Figure 9 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(cellular['Porosity'], cellular['Normalized strength (compression)'])
plt.plot(brittle['Porosity'], brittle['Normalized strength (compression)'])

ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_ylabel('Compressive Strength (normalized)')
plt.ylim(0.002, 1)
plt.xlim(20, 100)

plt.tight_layout()
fig.savefig('../figures/figure9.png')

In [ ]:
# Extract the bib ref of the above plot
dataplot = cellular[(cellular['Normalized strength (compression)'] > 0)]
authors = dataplot['Mendeley key']

print "cellular:"
list_authors(authors)

dataplot = brittle[(brittle['Normalized strength (compression)'] > 0)]
authors = dataplot['Mendeley key']

print "brittle:"
list_authors(authors)

# Compressive strength in presence and absence of ice-lenses type structural defects
Figure 11 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(notsureifdefects['Porosity'], notsureifdefects['Normalized strength (compression)'], color='white')
plt.plot(nodefects['Porosity'], nodefects['Normalized strength (compression)'])
plt.plot(defects['Porosity'], defects['Normalized strength (compression)'])
ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_ylabel('Compressive Strength (normalized)')
plt.ylim(0.001, 3)
plt.xlim(20, 100)

plt.tight_layout()

fig.savefig('../figures/figure11.png')

In [ ]:
# Extract the bib ref of the above plot
dataplot = defects[(defects['Normalized strength (compression)'] > 0)]
authors = dataplot['Mendeley key']
print "Defects: "
list_authors(authors)

dataplot = nodefects[(nodefects['Normalized strength (compression)'] > 0)]
authors = dataplot['Mendeley key']
print "No defects: "
list_authors(authors)

dataplot = notsureifdefects[(notsureifdefects['Normalized strength (compression)'] > 0)]
authors = dataplot['Mendeley key']
print "Not sure if defects: "
list_authors(authors)

# Influence of the starting particle size over the compressive strength
Figure 12 of the paper

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(data['Granulo poudre'], data['Normalized strength (compression)'])
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Particle size (\mu m)')
ax.set_ylabel('Strength (normalized)')
plt.ylim(0.001, 10)
plt.tight_layout()

# line with -1/2 slope
poresize_table = linspace(10, 50.0, num=10000)
plt.plot(poresize_table, 20*poresize_table**(-1/2), marker='', linestyle='-', color='black', linewidth='0.3')
plt.plot([20, 40], [1, 1], 'k-', lw=0.3)
plt.annotate('-1/2', (20, 1.2), fontsize='5')

fig.savefig('../figures/figure12.png')

In [ ]:
# Extract the bibliographic references used for the plot
dataplot = data[(data['Granulo poudre'] > 0) & (data['Normalized strength (compression)']>0)]
authors = dataplot['Mendeley key']
list_authors(authors)

# Influence of the nature of the solvent over the compressive strength.
Figure 13 of the paper

In [ ]:
fig = plt.figure(figsize = (3*figuresize, figuresize))

ax = fig.add_subplot(1,3,1)
plt.scatter(water['Porosity'], water['Normalized strength (compression)'], s=water['Pore width'], lw=0, alpha=0.3, color='#3778bf')
ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_ylabel('Compressive Strength (normalized)')
plt.title('Water', fontsize=titlefontsize, weight=titleweight)
plt.ylim(0.001, 2)
plt.xlim(20, 100)

ax = fig.add_subplot(1,3,2)
plt.scatter(camphene['Porosity'], camphene['Normalized strength (compression)'], s=camphene['Pore width'], lw=0, alpha=0.3, color='#3778bf')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_yticklabels([])
plt.title('Camphene', fontsize=titlefontsize, weight=titleweight)
plt.ylim(0.001, 2)
plt.xlim(20, 100)

ax = fig.add_subplot(1,3,3)
plt.scatter(tba['Porosity'], tba['Normalized strength (compression)'], s=tba['Pore width'], lw=0, alpha=0.3, color='#3778bf')
ax.set_yscale('log')
ax.set_xlabel('Porosity (\%)')
ax.set_yticklabels([])
plt.title('TBA', fontsize=titlefontsize, weight=titleweight)
plt.ylim(0.001, 2)
plt.xlim(20, 100)

plt.tight_layout()

fig.savefig('../figures/figure13.png')

In [ ]:
# Extract the bibliographic references used for the plot
dataplot = water[(water['Normalized strength (compression)']>0)]
authors = dataplot['Mendeley key']
print "water: "
list_authors(authors)

dataplot = camphene[(camphene['Normalized strength (compression)']>0)]
authors = dataplot['Mendeley key']
print "camphene: "
list_authors(authors)

dataplot = tba[(tba['Normalized strength (compression)']>0)]
authors = dataplot['Mendeley key']
print "tba: "
list_authors(authors)

# Compressive strength: measured (all materials) vs. predicted by Ashby honeycomb out-of-plane model
Figure 16 of the paper

In [ ]:
# Multipannels figure

# value for the 1:1 line on the plots
strength_table = linspace(0.1, 1000.0, num=10000)
        
def plot_upper_limit_honeycomb():
    plt.plot(strength_table, strength_table, linestyle='--', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round', markevery='2')    

f,  (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=True, figsize=(figuresize, 2*figuresize))
plt.ylim(0.11, 2000)

ax1.plot(water['Strength (MPa)'], water['Honeycomb_Ashby'])
ax1.plot(defects['Strength (MPa)'], defects['Honeycomb_Ashby'])
ax1.plot(strength_table, strength_table, linestyle='--', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round', markevery='2')
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_ylabel(' ')
closed_text = ax1.annotate("Water", xy=(0.2,600), fontsize=titlefontsize, color='k')

ax2.plot(camphene['Strength (MPa)'], camphene['Honeycomb_Ashby'])
ax2.plot(strength_table, strength_table, linestyle='--', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round', markevery='2')
plot_upper_limit_honeycomb()
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.set_ylabel('Predicted strength (honeycomb model) (MPa)')
closed_text = ax2.annotate("Camphene", xy=(0.2,600), fontsize=titlefontsize, color='k')

ax3.plot(tba['Strength (MPa)'], tba['Honeycomb_Ashby'])
ax3.plot(strength_table, strength_table, linestyle='--', linewidth='0.5', marker='', dashes=[2, 2], color='0.5', zorder=1, dash_capstyle='round', markevery='2')
plot_upper_limit_honeycomb()
ax3.set_yscale('log')
ax3.set_xscale('log')
ax3.set_ylabel(' ')
ax3.set_xlabel('Measured compressive strength (MPa)')
closed_text = ax3.annotate("Tert-butyl alcohol", xy=(0.2,600), fontsize=titlefontsize, color='k')

plt.tight_layout()
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

f.savefig('../figures/figure16.png')

In [ ]:
# Extract the bib ref of the above plot
data_water = water[(water['Strength (MPa)'] > 0) & (water['Honeycomb_Ashby'] > 0)]
data_camphene = camphene[(camphene['Strength (MPa)'] > 0) & (camphene['Honeycomb_Ashby'] > 0)]
data_tba = tba[(tba['Strength (MPa)'] > 0) & (tba['Honeycomb_Ashby'] > 0)]

authors = data_water['Mendeley key']
print 'Water: '
list_authors(authors)

authors = data_camphene['Mendeley key']
print 'Camphene: '
list_authors(authors)

authors = data_tba['Mendeley key']
print 'TBA: '
list_authors(authors)

In [ ]:
#%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
# you need to run the line above only once
%load_ext version_information
%reload_ext version_information

%version_information numpy, scipy, matplotlib

# Acknowledgements
The research leading to these results has received funding from the European Research Council under the European Community's Seventh Framework Programme (FP7/2007-2013) Grant Agreement no. 278004 (project FreeCo)